Vamos a ilustrar como paralelizar bucles for en Python con un ejemplo:

La idea es paralelizar el siguiente algoritmo KNN para clasificacion basado en la distancia de Gower-BQ (una variante de la distancia de Gower aplicable a conjunto de datos binarios-cuantitativos).

Aqui lo importante no es el funcionamiento del algoritmo en si, si no que hay una parte de este que genera unos costes computacionales altos, lo que se traduce en que al aplicar el algoritmo a un data-set de 5000 variables, este tarde mas de 13 minutos en compilarse por completo. La idea de paralelizarlo es reducir este tiempo de cómputo, para asi hacerlo mas aplicable en contextos reales.

El algoritmo original (sin paralelizar):

In [1]:
import pandas as pd
import numpy as np

In [2]:
def KNN_classification( X , Y , x_new, k, distance = "Gower-BQ" ,  p1=0, p2=0 ):

 
####################################################################################################################################################################################################################################################

    X = pd.concat([X, x_new.to_frame().T], ignore_index=True)

    distances = []

    groups_knn = []

######################################################################################################################################


    if distance == "Gower-BQ":

        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def GowerBQ_Similarity_Python(i,j, BQ_Data_Set, p1, p2):

            X = BQ_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical 

##########################################################################################
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0, p1)

            for r in range(0, p1): 

                G_vector[r] = G(r, X)
##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
         
 
##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] 
     
            numerator = numerator_part_1 + numerator_part_2

            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1])  

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################


        def Dist_GowerBQ_Py(i, j, BQ_Data ,  p1, p2):

            Dist_Gower = np.sqrt( 1 - GowerBQ_Similarity_Python(i, j, BQ_Data , p1, p2) )

            return(Dist_Gower)

##########################################################################################

        for i in range(1, len(X)):

            distances.append( Dist_GowerBQ_Py( len(X), i , X, p1, p2) )


######################################################################################################################################

######################################################################################################################################
    
    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        groups_knn.append(Y.iloc[i,:])

    unique, counts = np.unique(groups_knn , return_counts=True)

    unique_Y , counts_Y = np.unique(Y , return_counts=True)

    if len(unique) == len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique_Y })
    
    elif len(unique) < len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique })



    prediction_group = proportions_groups_knn.sort_values(by=["proportions_groups"], ascending=False).iloc[0,:]['groups']

    message = print( "x_new is classify in the group", prediction_group , ". So KNN algorithm predict y_new =",  prediction_group )                                      
                                       

    return proportions_groups_knn , message  

Vamos a probar el algoritmo con un mismo data-set pero con distintos tamaños de filas (observaciones).

Preparamos los datos para la prueba:

In [3]:
Gender_classification = pd.read_csv('gender_classification.csv')

x_new = pd.Series({'long_hair': 1, 'forehead_width_cm': 4, 'forehead_height_cm': 6, 'nose_wide': 1 , 'nose_long': 1 , 'nose_long': 1 , 'lips_thin':1, 'distance_nose_to_lip_long': 1 })

Y = Gender_classification.loc[: , ['gender']]

from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()

Y['gender'] = ord_enc.fit_transform(Y[['gender']])

X = Gender_classification.loc[: , ['forehead_width_cm', 'forehead_height_cm',   # Quantitative (2)

                   'long_hair', 'nose_wide', 'nose_long', 'lips_thin', 'distance_nose_to_lip_long'     # Binary (5)
                 
                            ]] 


In [4]:
X_small_1 = X.iloc[0:1000,:]
Y_small_1 = Y.iloc[0:1000,:]

X_small_2 = X.iloc[0:2000,:]
Y_small_2 = Y.iloc[0:2000,:]

X_small_3 = X.iloc[0:3000,:]
Y_small_3 = Y.iloc[0:3000,:]

In [5]:
KNN_classification( X_small_1 , Y_small_1  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

20seg / 27seg

In [6]:
KNN_classification( X_small_2 , Y_small_2  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

2.47min  / 2.35

In [7]:
KNN_classification( X_small_3 , Y_small_3  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

8.51min

In [8]:
KNN_classification( X , Y  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

KeyboardInterrupt: 

In [ ]:
from joblib import Parallel, delayed
import multiprocessing


In [ ]:
def KNN_classification_parallel( X , Y , x_new, k, distance = "Gower-BQ" ,  p1=0, p2=0 ):

 
####################################################################################################################################################################################################################################################

    X = pd.concat([X, x_new.to_frame().T], ignore_index=True)

    distances = []

    groups_knn = []

######################################################################################################################################


    if distance == "Gower-BQ":

        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def GowerBQ_Similarity_Python(i,j, BQ_Data_Set, p1, p2):

            X = BQ_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical 

##########################################################################################
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0, p1)

            for r in range(0, p1): 

                G_vector[r] = G(r, X)
##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
         
 
##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] 
     
            numerator = numerator_part_1 + numerator_part_2

            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1])  

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################

        def Dist_GowerBQ_Py(i, j, BQ_Data ,  p1, p2):

            Dist_Gower = np.sqrt( 1 - GowerBQ_Similarity_Python(i, j, BQ_Data , p1, p2) )

            return(Dist_Gower)

##########################################################################################

# ESTA ES LA PARTE CRITICA DEL ALGORITMO, LA QUE LO HACE LENTO. ESTE BUCLE FOR ES EL QUE DEBEMOS PARALELIZAR.


        # for i in range(1, len(X)):

           # distances.append( Dist_GowerBQ_Py( len(X), i , X, p1, p2) )

        n_jobs  = multiprocessing.cpu_count()


        distances = Parallel(n_jobs=n_jobs)( delayed(Dist_GowerBQ_Py)( len(X), i , X, p1, p2) for i in range(1, len(X)) )


######################################################################################################################################

######################################################################################################################################
    
    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        groups_knn.append(Y.iloc[i,:])

    unique, counts = np.unique(groups_knn , return_counts=True)

    unique_Y , counts_Y = np.unique(Y , return_counts=True)

    if len(unique) == len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique_Y })
    
    elif len(unique) < len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique })



    prediction_group = proportions_groups_knn.sort_values(by=["proportions_groups"], ascending=False).iloc[0,:]['groups']

    message = print( "x_new is classify in the group", prediction_group , ". So KNN algorithm predict y_new =",  prediction_group )                                      
                                       

    return proportions_groups_knn , message  

Probamos ahora el algoritmo paralelizado con los mismos ejemplos que antes:

In [ ]:
KNN_classification_parallel( X_small_1 , Y_small_1  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

15.5 seg vs 20seg

In [ ]:
KNN_classification_parallel( X_small_2 , Y_small_2  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

1min vs 2.47min

In [ ]:
KNN_classification_parallel( X_small_3 , Y_small_3  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

x_new is classify in the group 1.0 . So KNN algorithm predict y_new = 1.0


(   proportions_groups  groups
 0                 1.0     1.0,
 None)

3.33min vs 8.51min

In [ ]:
KNN_classification_parallel( X , Y  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

KeyboardInterrupt: 

https://www.cienciadedatos.net/documentos/py12-paralelizar-con-python.html